In [2]:
import numpy as np
import pandas as pd

## Covariance

In [5]:
s1 = pd.util.testing.makeTimeSeries(7)
s2 = s1 + np.random.randn(len(s1)) / 10
print(s1)
print(s2)

2000-01-03   -0.184459
2000-01-04    0.061743
2000-01-05   -0.121739
2000-01-06   -0.195322
2000-01-07   -0.682264
2000-01-10   -0.989385
2000-01-11   -0.281997
Freq: B, dtype: float64
2000-01-03   -0.192549
2000-01-04    0.235489
2000-01-05   -0.149195
2000-01-06   -0.056964
2000-01-07   -0.536852
2000-01-10   -1.113184
2000-01-11   -0.257206
Freq: B, dtype: float64


In [6]:
s1.cov(s2)

0.15193219594720536

In [7]:
d1 = pd.util.testing.makeTimeDataFrame()
print (d1.head())
print (d1.cov())

                   A         B         C         D
2000-01-03 -1.049066 -1.087674 -0.031728  0.026687
2000-01-04  1.692604 -0.553027 -0.120174  0.439875
2000-01-05  1.391788 -0.080056  0.359769 -1.226489
2000-01-06 -0.064914  0.581360 -1.213542 -0.358923
2000-01-07 -0.270513  0.286368  0.062560 -1.654739
          A         B         C         D
A  1.232334 -0.028421  0.455840 -0.105892
B -0.028421  1.120736 -0.383459 -0.311969
C  0.455840 -0.383459  0.939437  0.188690
D -0.105892 -0.311969  0.188690  0.944081


##  Correlation

In [8]:
s1.corr(s2, method='pearson')

0.97435417348463105

In [9]:
d1.corr()

,A,B,C,D
A,1.000000,-0.024183,0.423657,-0.098173
B,-0.024183,1.000000,-0.373709,-0.303288
C,0.423657,-0.373709,1.000000,0.200360
D,-0.098173,-0.303288,0.200360,1.000000


## Rolling 

In [10]:
[n for n in dir(pd) if n.startswith('rolling')]

['rolling_apply',
 'rolling_corr',
 'rolling_count',
 'rolling_cov',
 'rolling_kurt',
 'rolling_max',
 'rolling_mean',
 'rolling_median',
 'rolling_min',
 'rolling_quantile',
 'rolling_skew',
 'rolling_std',
 'rolling_sum',
 'rolling_var',
 'rolling_window']

Let's see some examples:

In [11]:
s3 = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
s3 = s3.cumsum()
s3_max = s3.rolling(60).max()
s3_mean = s3.rolling(60).mean()
s3_min = s3.rolling(60).min()
data = {'cumsum':s3, 'max':s3_max, 'mean':s3_mean, 'min':s3_min}
df = pd.DataFrame(data)
df.tail()

,cumsum,max,mean,min
2002-09-22,44.604940,55.916173,50.230014,42.028870
2002-09-23,43.531506,55.916173,50.083489,42.028870
2002-09-24,44.541476,55.916173,49.931458,42.028870
2002-09-25,42.656658,55.916173,49.746954,42.028870
2002-09-26,41.416085,55.916173,49.544477,41.416085


In [13]:
import bokeh.plotting as bk
bk.output_notebook()

Loading BokehJS ...

In [14]:
fig = bk.figure(x_axis_type = "datetime",
               tools="pan,box_zoom,reset", title = 'Rolling Moments',
               plot_width=750, plot_height=400)
fig.line(df.index, df['cumsum'], color='cadetblue', legend='Cumulative Sum')
fig.line(df.index, df['max'], color='mediumorchid', legend='Max')
fig.line(df.index, df['min'], color='mediumpurple', legend='Min')
fig.line(df.index, df['mean'], color='navy', legend='Min')
bk.show(fig)

In [15]:
s4 = s3 + np.random.randn(len(s3))
rollc = s3.rolling(window=10).corr(s3)
data2 = {'cumsum':s3, 'similar':s4, 'rolling correlation':rollc}
df2 = pd.DataFrame(data2)

In [16]:
fig = bk.figure(x_axis_type = "datetime", title = 'Rolling Correlation',
       plot_width=750, plot_height=400)
fig.line(df2.index, df2['cumsum'], color='cadetblue', legend='Cumulative Sum')
fig.line(df2.index, df2['similar'], color='mediumorchid', legend='Similar')
fig.line(df2.index, df2['rolling correlation'], color='navy', legend='Rolling Corr.')
fig.legend.location = "bottom_right"
bk.show(fig)

## example

In [20]:
AAPL = pd.read_csv('example_data/p03_AAPL.txt', index_col='Date', parse_dates=True)
price = AAPL['Adj Close']
AAPL.tail(5)

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2012-09-17,699.35,699.80,694.61,699.78,14215400,699.78
2012-09-18,699.88,702.33,696.42,701.91,13339400,701.91
2012-09-19,700.26,703.99,699.57,702.10,11674100,702.10
2012-09-20,699.16,700.06,693.62,698.70,12020300,698.70
2012-09-21,702.41,705.07,699.36,700.09,20393600,700.09


In [23]:
returns = price.pct_change()
ret_index = (1 + returns).cumprod()
ret_index[0] = 1
monthly_returns = ret_index.resample('BM').last().pct_change()

In [24]:
fig = bk.figure(x_axis_type = 'datetime', title = 'Monthly Returns',
                plot_width=750, plot_height=400)
fig.line(monthly_returns.index, monthly_returns)
bk.show(fig)